<a href="https://colab.research.google.com/github/vmatiasw/modelos_y_simulacion/blob/main/P4E6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
from collections import Counter
import random
import time
import numpy as np
import math

In [88]:
def medir_tiempo(func):
    inicio = time.perf_counter()
    resultado = func()
    duracion = time.perf_counter() - inicio
    return resultado, duracion

def calcular_TVD(dict_p1, dict_p2):
    """Distancia Total de Variación (TVD) entre dos distribuciones."""
    soporte_total = set(dict_p1) | set(dict_p2)
    return 0.5 * sum(abs(dict_p1.get(k, 0) - dict_p2.get(k, 0)) for k in soporte_total)

def calcular_momentos(dict_fpm):
    """Calcula esperanza, varianza y desviación estándar de una distribución discreta."""
    esperanza = sum(x * p for x, p in dict_fpm.items())
    varianza = sum((x - esperanza)**2 * p for x, p in dict_fpm.items())
    return esperanza, varianza, varianza**0.5

def comparar_distribuciones_df(p_estimada, p_real):
    soporte = list(p_real.keys())
    data = [{
        "Datos": f"P({x})",
        "Real": p_real.get(x, 0),
        "Estimación": p_estimada.get(x, 0),
        "Diferencia": abs(p_real.get(x, 0) - p_estimada.get(x, 0))
    } for x in soporte]

    esp_r, var_r, desv_r = calcular_momentos(p_real)
    esp_e, var_e, desv_e = calcular_momentos(p_estimada)

    resumen = [
        {"Datos": "Esperanza", "Real": esp_r, "Estimación": esp_e, "Diferencia": abs(esp_r - esp_e)},
        {"Datos": "Varianza", "Real": var_r, "Estimación": var_e, "Diferencia": abs(var_r - var_e)},
        {"Datos": "Desv. Est.", "Real": desv_r, "Estimación": desv_e, "Diferencia": abs(desv_r - desv_e)},
    ]
    return pd.concat([pd.DataFrame(data), pd.DataFrame(resumen)], ignore_index=True)

def calificar_generador_de_muestras(generador, dict_p_real, n=100_000):
  soporte = list(dict_p_real.keys())

  muestras, tiempos = zip(*(medir_tiempo(generador) for _ in range(n)))
  frec = Counter(muestras)
  dict_p_estimada = {x: frec.get(x, 0) / n for x in soporte}

  df_comparacion = comparar_distribuciones_df(dict_p_estimada, dict_p_real)
  tvd = calcular_TVD(dict_p_estimada, dict_p_real)
  t_prom = np.mean(tiempos)

  print(df_comparacion.to_string(index=False))
  print()
  print(f"Distancia total de variación: {tvd:.6f}")
  print(f"Tiempo promedio de ejecución: {t_prom} segundos")


In [79]:
DICT_FPM = {i+1: p for i, p in enumerate([0.15, 0.20, 0.10, 0.35, 0.20])}
SOPORTE = list(DICT_FPM.keys())
print('funcion de probabilidad de masa: ',DICT_FPM)
print('debe dar 1: ',sum(DICT_FPM.values()))

funcion de probabilidad de masa:  {1: 0.15, 2: 0.2, 3: 0.1, 4: 0.35, 5: 0.2}
debe dar 1:  1.0


In [90]:
# I) Describir mediante un pseudocódigo un algoritmo que simule X utilizando el
# método de la transformada inversa y que minimice el número esperado de búsquedas.

def ordenar_dict_fpm(dict_fpm):
  return dict(sorted(dict_fpm.items(), key=lambda x: x[1], reverse=True))

def calcular_dict_fda(dict_fpm):
  """
  Recibe una funcion de probabilidad de masa en forma de diccionario de pares
  (valor, probabilidad) y devuelve su funcion de distribucion acumulada.
  Solo definidas para el soporte.
  """
  F = {}
  suma = 0
  for valor, probabilidad in dict_fpm.items():
    suma += probabilidad
    F[valor] = suma
  return F

def TI(dict_fda):
  u = random.random()
  for v, p in dict_fda.items():
    if u < p:
      return v

print('Sin ordenar:')
dict_fda = calcular_dict_fda(DICT_FPM)
generador_de_muestras = lambda: TI(dict_fda)
calificar_generador_de_muestras(generador_de_muestras, DICT_FPM)
print()
print('Ordenando:')
dict_fda = calcular_dict_fda(ordenar_dict_fpm(DICT_FPM))
generador_de_muestras = lambda: TI(dict_fda)
calificar_generador_de_muestras(generador_de_muestras, DICT_FPM)

Sin ordenar:
     Datos     Real  Estimación  Diferencia
      P(1) 0.150000    0.151530    0.001530
      P(2) 0.200000    0.200490    0.000490
      P(3) 0.100000    0.098900    0.001100
      P(4) 0.350000    0.350210    0.000210
      P(5) 0.200000    0.198870    0.001130
 Esperanza 3.250000    3.244400    0.005600
  Varianza 1.887500    1.892569    0.005069
Desv. Est. 1.373863    1.375707    0.001843

Distancia total de variación: 0.002230
Tiempo promedio de ejecución: 1.1496690197236604e-06 segundos

Ordenando:
     Datos     Real  Estimación  Diferencia
      P(1) 0.150000    0.151850    0.001850
      P(2) 0.200000    0.199770    0.000230
      P(3) 0.100000    0.098980    0.001020
      P(4) 0.350000    0.350650    0.000650
      P(5) 0.200000    0.198750    0.001250
 Esperanza 3.250000    3.244680    0.005320
  Varianza 1.887500    1.892952    0.005452
Desv. Est. 1.373863    1.375846    0.001983

Distancia total de variación: 0.002500
Tiempo promedio de ejecución: 2.448607080

In [85]:
# II) Describir mediante un pseudocódigo un algoritmo que simule X utilizando el método de aceptación y
# rechazo con una variable soporte Y con distribución binomial B(4,0.45).
N_BIN = 4
P_BIN = 0.45

def crear_dict_fpm(soporte, p):
  return {v: p(v) for v in soporte}

def adecuar_soporte(dict_fpm_objetivo, dict_fpm_modelo):
    soporte_modelo = dict_fpm_modelo.keys()
    return {nuevo_k: v for nuevo_k, v in zip(soporte_modelo, dict_fpm_objetivo.values())}

def fpm_binomial(k, n=N_BIN, p=P_BIN):
    if not 0 <= k <= n:
        return 0.0
    coef = math.comb(n, k)
    prob = coef * (p ** k) * ((1 - p) ** (n - k))
    return prob

_dict_fpm_binomial = crear_dict_fpm(list(range(N_BIN+1)), fpm_binomial)
dict_fpm_binomial = adecuar_soporte(_dict_fpm_binomial, DICT_FPM)

print('debe dar 1: ',sum(dict_fpm_binomial.values())) # da cerca por la precision flotante de python
print()
print(f'X~B({N_BIN},{P_BIN})')
for i in range(1,6):
  print(f'P(X={i}) = ', dict_fpm_binomial[i])

print()
print('funcion de probabilidad de masa de p: ',DICT_FPM)

debe dar 1:  1.0000000000000002

X~B(4,0.45)
P(X=1) =  0.09150625000000003
P(X=2) =  0.2994750000000001
P(X=3) =  0.3675375000000001
P(X=4) =  0.20047500000000004
P(X=5) =  0.04100625

funcion de probabilidad de masa de p:  {1: 0.15, 2: 0.2, 3: 0.1, 4: 0.35, 5: 0.2}


In [86]:
# Método de rechazo con una uniforme discreta, buscando la cota c más baja posible
# con las probabilidades dadas.

C_posibles = [DICT_FPM[i] / dict_fpm_binomial[i] for i in range(1,6)]
C = min(C_posibles)

print(C_posibles)
print('C = ',C)

[1.6392322928761691, 0.6678353785791801, 0.2720810801618882, 1.745853597705449, 4.877305288827923]
C =  0.2720810801618882


In [93]:
def AyR(p, q, c=C):
  """
  Método de aceptación-rechazo para generar muestras de la distribución `p` usando una distribución propuesta `q` y una constante `c`.

  Parámetros:
  p (list/dict): Distribución de probabilidad, donde `p[y]` es la probabilidad asociada a un valor `y`.
  q (function): Función que genera un valor aleatorio `y` según la distribución propuesta.
  c (float): Constante de escala que ajusta la relación entre `p` y `q`.

  Devuelve:
  int: Un valor `y` del soporte de `p` basado en la probabilidad ajustada por `q` y `c`.
  """
  while True:
    u = random.random()
    y = q()
    qy = dict_fpm_binomial[y]
    py = p[y]
    if u < py/(qy*c):
      return y

def q():
  u = random.random()
  indice_random = int(u*len(SOPORTE))
  return SOPORTE[indice_random]

generador_de_muestras = lambda: AyR(DICT_FPM, q)
calificar_generador_de_muestras(generador_de_muestras, DICT_FPM)

     Datos     Real  Estimación  Diferencia
      P(1) 0.150000    0.197990    0.047990
      P(2) 0.200000    0.199850    0.000150
      P(3) 0.100000    0.199790    0.099790
      P(4) 0.350000    0.199240    0.150760
      P(5) 0.200000    0.203130    0.003130
 Esperanza 3.250000    3.009670    0.240330
  Varianza 1.887500    2.003476    0.115976
Desv. Est. 1.373863    1.415442    0.041579

Distancia total de variación: 0.150910
Tiempo promedio de ejecución: 1.0611686494303285e-06 segundos
